In [ ]:
%matplotlib inline
from importlib import reload
import data_analysis as da
import run_lichee as rl
import inference as inf
import numpy as np
reload(inf)
reload(da)
reload(rl)

In [ ]:
import subprocess
path = "hossein/exp1.tsv"
print("Parsing Data...")
panel,sample_names = da.parse_data(path)
#print(panel.axes)
panel["major"] = panel["major"].astype(np.int64)
#data,_ = generate_data()
print("Copying Data...")
subprocess.call(["rm","tmp.hd5"])
subprocess.check_call(["cp","hossein/trace1.hd5","tmp.hd5"])
print("Generating Model...")
model,trace = inf.build_model(panel,10000,3300,"tmp.hd5")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
#pprint(trace.stat_names)
print(trace[200]["location_indicies"])
#plt.plot(trace[30:]["cluster_clustering"])


In [ ]:
raise Exception()
l = []
for i in range(15):
    model,trace = inf.build_model(panel,44000,5000,"traces/trace_{}.pkl".format(i),cluster_params="one")
    x = np.mean(trace["logP"][3000:]),np.std(trace["logP"][3000:])
    print(x)

In [ ]:
import seaborn as sns
import gc
BURNIN=4000
SUBSAMPLE=50
with open("/scratch/chuebner/Biclustering/lichee_out/lichee_1/c_count.txt","r") as f:
    lines = f.read().split("\n")
    good = []
    bad = []
    sub_trace = trace[BURNIN::SUBSAMPLE]
    for line in lines[:-1]:
        print(line)
        a,b,c = line.split(":")
        index,count,max_n = int(a),int(b),int(c)
        i = (index-BURNIN)/SUBSAMPLE+1
        if (count==max_n):
            good.append(dict(sub_trace[i]))
            print(line,"--------------")
        else:
            bad.append(dict(sub_trace[i]))
            print(line)


In [ ]:
def count_unique(x):
    TRESH = 2
    ci = x["cluster_indicies"]
    pli = x["location_indicies"]
    pac,counts = np.unique(ci,return_counts=True)
    ac = pac[counts > TRESH]
    aci = ci[ac,:]
    total_active_unique_axis_clusters = np.unique(aci+np.arange(aci.shape[1])*14).shape[0]
    return total_active_unique_axis_clusters

index = sorted(range(len(sub_trace)),key=lambda k: count_unique(sub_trace[k]))
[(count_unique(sub_trace[x]),x) for x in index]

In [ ]:
import seaborn as sns
sns.distplot(np.array(trace["logP"][500:2600])-np.array(trace["logP"][501:1901]))
print(np.mean(trace[""][500:2100]))

In [ ]:
import matplotlib.pyplot as plt
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()

N = 1
mask = np.ones((N,))/N
f = lambda x: np.convolve(x, mask, mode='valid')
ax1.plot(f(list(-trace["logP"][50:])), 'g-')
ax2.plot(f(list(trace["cluster_clustering"][50:])), 'b-')
#print(trace["axis_cluster"][26000])

ax1.set_xlabel('X data')
ax1.set_ylabel('log probablity of a configuration', color='g')
ax2.set_ylabel('cluster concentration parameter', color='b')


In [ ]:
print(panel.axes[0])
print(panel["total_raw_e"])

In [ ]:
import seaborn
seaborn.set()
from pprint import pprint
import pandas as pd

pos = 200
panel,sample_names = da.parse_data(path)
df = panel.to_frame()
clustering = trace[pos]["location_indicies"]
#curr_tcns = trace[pos]["tcns"]
pd.options.display.max_columns = 500
#df = df[["sample_id","Origin Node","alt_counts","ref_counts"]]
panel["vaf"] = panel["alt_counts"]/(panel["alt_counts"]+panel["ref_counts"])
#panel = da.add_column_to_panel(panel,"inferred_cn",curr_tcns)
#tcns = panel["inferred_cn"]
tcnt = trace[pos]["tumour_content"]
tre = panel["total_raw_e"]
#panel["ccf"] = panel["vaf"] * (2*(1-tcnt)+tre*tcnt)/(tcns*tcnt)
#df = df[["Origin Node","y"]]
pn = da.add_column_to_panel(panel,"cluster_id",clustering)
#print(pn["cluster_id"])
df = pn.to_frame()
df = df.reset_index()
#df = df[df["cluster_id"].isin([5,6])] 
seaborn.factorplot(
    #hue='cluster_id',
    #hue='event_id',
    #dodge='True',
    y='vaf', x='sample_id', #row='cluster_id',
    data=df, 
    kind='strip',
    jitter=True,alpha=0.5,
    aspect=4,size=4)
print(trace["cluster_magnitudes"][pos])

In [ ]:
counts = df.loc[df["cluster_id"] == 0,["alt_counts","ref_counts","sample_id","event_id"]].set_index(["event_id","sample_id"]).unstack()
print(counts)

In [ ]:
pos = 18000
maj = df.loc[df["cluster_id"] == 2,["major","sample_id","event_id"]].set_index(["event_id","sample_id"]).unstack()
tcn = pd.DataFrame(trace["tcns"][pos],index=pn.axes[1],columns=pn.axes[2]).loc[maj.index]

print(tcn/maj)


In [ ]:
cluster_locations = inf.get_map_item(model,trace,"cluster_locations")
fig = plt.figure(figsize=(10, 100))
ax = plt.gca()
seaborn.heatmap(cluster_locations, ax=ax, cmap="YlGnBu")

In [ ]:
stat = trace[18000]["tumour_content"]
print(stat)

In [ ]:
import pandas as pd
pos = 10100
cluster_locations = trace[-1]["cluster_locations"]
#cluster_locations = trace[200]["cluster_indicies"]
location_indicies = trace[-1]["location_indicies"]
columns=da.get_sample_names(pd.read_csv(path,sep="\t"))

entries = {}
#count datapoints in each cluster and add 
for i in location_indicies:
    if i in entries:
         entries[i]["count"] += 1
    else:
        entry = {columns[0][j]:cluster_locations[i,j] for j in range(columns[1])}
        entry["cluster_index"] = i
        entry["count"] = 1
        entries[i] = entry
df = pd.DataFrame(list(entries.values()))
df.sort_values("count",ascending=False,inplace=True)
print(df)
    

In [ ]:
 pos = -1
location_indicies = trace[pos]["location_indicies"]
cluster_indicies = trace[pos]["cluster_indicies"]
axis_cluster_locations = trace[pos]["axis_cluster_locations"]
cluster_clustering = None#inf.get_map_item(model,trace,"cluster_clustering")
ssnv_input,cluster_input = rl.build_lichee_inputs("",panel,location_indicies,cluster_indicies,axis_cluster_locations,5)